# Прогнозирование. Модели.

> 🚀 В этой практике нам понадобятся: `etna==2.10.0, numpy==1.26.4, pandas==1.5.3, matplotlib==3.10.3, seaborn==0.13.2, lightgbm==4.6.0, prophet==1.1.6` 

> 🚀 Установить вы их можете с помощью команды: `%pip install etna==2.10.0 numpy==1.26.4 pandas==1.5.3 matplotlib==3.10.3 seaborn==0.13.2 lightgbm==4.6.0 prophet==1.1.6` 


## Содержание

* [Загрузка выборок](#Загрузка-выборок)
* [Статистические модели](#Статистические-модели)
* [Модели регрессоры](#Модели-регрессоры)
  * [Генерация признаков](#Генерация-признаков)
  * [Линейная регрессия](#Линейная-регрессия)
  * [LGBM : Бустинг](#LGBM-:-Бустинг)
* [Заключение](#Заключение)
* [Вопросы для закрепления](#Вопросы-для-закрепления)


Привет! В предыдущем ноутбуке мы подготовили выборки, на которых теперь будем учить и тестировать модели. 

In [227]:
from datetime import date, timedelta
from pathlib import Path

import warnings
warnings.filterwarnings("ignore")

import ipywidgets as widgets
from matplotlib import pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")

from etna.datasets.tsdataset import TSDataset
from etna.models import HoltWintersModel
from etna.pipeline import Pipeline

import numpy as np 
import pandas as pd 

from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression
from prophet.make_holidays import make_holidays_df

## Загрузка выборок

Для начала загрузим ранее подготовленные выборки.

In [228]:
dpath = Path().cwd() / "ts_datasets"

train_fpath = dpath / "train.csv"
train_df = pd.read_csv(train_fpath, parse_dates=["timestamp"], index_col=0)

test_fpath = dpath / "test.csv"
test_df = pd.read_csv(test_fpath, parse_dates=["timestamp"], index_col=0)

train_df.shape, test_df.shape

((222288, 5), (2728, 5))

В дальнейшем нам также понадобится информация о: 
- горизонте прогнозирования (в нашем случае хотим на 31 день);
- дата разделения данных (1 декабря 2016г).

Вынесем их в отдельные переменные, чтобы меньше хардкодить. 

In [229]:
HORIZON = 31
SPLIT_DATE = date(2016, 12, 1)

## Статистические модели

Начнём со статистических моделей. Таким моделям по сути не нужно обучение, т.к. они используют чисто статистические зависимости, на базе которых генерируют предсказания в будущее. 

В этот раз мы посмотрим на модель Хольта-Уинтерса. Звучит пафосно, но под красивым названиям скрывается экспоненциальное сглаживание. 

Эта модель уже есть в ETNA, давайте ею и воспользуемся.

Так как работаем с ETNA, снова конвертируем таблицы в ETNA-датасеты и имитируем `segment` для нашего временного ряда. Сделаем это как в обучающей выборке, так и в тестовой. 

In [230]:
train_ts_df = train_df.copy(deep=True)
test_ts_df = test_df.copy(deep=True)

In [231]:
# TODO - добавьте колонку segment в обучающую и тестовую выборки
#    объдините колонки country, store и product через нижнее подчёркивание (_)
train_ts_df["segment"] = train_ts_df["country"] + "_" + train_ts_df["store"] + "_" + train_ts_df["product"]
test_ts_df["segment"] = test_ts_df["country"] + "_" + test_ts_df["store"] + "_" + test_ts_df["product"]

assert "segment" in train_ts_df.columns, "Колонка `segment` не найдена"
assert "segment" in test_ts_df.columns, "Колонка `segment` не найдена"

print("Тесты прошли! Всё хорошо!")


Тесты прошли! Всё хорошо!


In [232]:
assert "segment" in train_ts_df.columns, "Колонка `segment` не найдена"
assert "segment" in test_ts_df.columns, "Колонка `segment` не найдена"

print("Тесты прошли! Всё хорошо!")

Тесты прошли! Всё хорошо!


In [233]:
# TODO - удалите колонки country, store и product
train_ts_df = train_ts_df.drop(columns=["country", "store", "product"])
test_ts_df = test_ts_df.drop(columns=["country", "store", "product"])

train_cols = train_ts_df.columns
test_cols = test_ts_df.columns

for col_name in ["country", "store", "product"]:
    assert col_name not in train_cols, f"В обучающей выборке всё ещё есть колонка {col_name}"
    assert col_name not in test_cols, f"В тестовой выборке всё ещё есть колонка {col_name}"

print("Тесты прошли! Всё хорошо!")


Тесты прошли! Всё хорошо!


In [234]:
train_cols = train_ts_df.columns
test_cols = test_ts_df.columns 

for col_name in ["country", "store", "product"]:
    assert col_name not in train_cols, f"В обучающей выборке всё ещё есть колонка {col_name}"
    assert col_name not in test_cols, f"В тестовой выборке всё ещё есть колонка {col_name}"

print("Тесты прошли! Всё хорошо!")

Тесты прошли! Всё хорошо!


In [ ]:
import pandas as pd
from etna.datasets.tsdataset import TSDataset


train_df["segment"] = train_df["country"] + "_" + train_df["store"] + "_" + train_df["product"]
test_df["segment"] = test_df["country"] + "_" + test_df["store"] + "_" + test_df["product"]


assert "segment" in train_df.columns, "train_df 缺少 segment 列"
assert "segment" in test_df.columns, "test_df 缺少 segment 列"



required_cols = ["timestamp", "segment", "target"]
assert all(col in train_df.columns for col in required_cols), "train_df 缺少必要列"
assert all(col in test_df.columns for col in required_cols), "test_df 缺少必要列"


train_wide_df = TSDataset.to_dataset(train_df)  
test_wide_df = TSDataset.to_dataset(test_df)


train_ts_df = TSDataset(df=train_wide_df, freq="D")
test_ts_df = TSDataset(df=test_wide_df, freq="D")



assert isinstance(train_ts_df, TSDataset), f"Неверный тип данных: {type(train_ts_df)}"
assert isinstance(test_ts_df, TSDataset), f"Неверный тип данных: {type(test_ts_df)}"

print("Тесты прошли! Всё хорошо!")

Тесты прошли! Всё хорошо!


In [236]:
assert isinstance(train_ts_df, TSDataset), f"Неверный тип данных: {type(train_ts_df)}"
assert isinstance(test_ts_df, TSDataset), f"Неверный тип данных: {type(test_ts_df)}"

print("Тесты прошли! Всё хорошо!")

Тесты прошли! Всё хорошо!


Датасеты сконвертированы, можно "учить" модель. 

Но на поверхности лежит вопрос: "А на чём учить, если признаков-то нет?". И это хороший вопрос. ETNA позволяет сгенерировать фичи прямо через интерфейс обучения модели.

В нашем случае добавим: 
- информацию о тренде в данных;
- информацию о сезонной составляющей.

<details>
    <summary>🤓 Напоминалка про тренды и сезонность [Нажми на меня]</summary>

**Тренд** - это какая-то общая зависимость, характерная для всего ряда целиком. **Сезонность** - это периодически повторяющаяся компонента. Обе эти составляющие часто присутствуют в ряде одновременно. 

</details>

In [237]:
model = HoltWintersModel(
    trend="add", 
    seasonal="add", 
    seasonal_periods=7,
)
pipeline = Pipeline(model=model, horizon=HORIZON)

pipeline.fit(train_ts_df)

Pipeline(model = HoltWintersModel(trend = 'add', damped_trend = False, seasonal = 'add', seasonal_periods = 7, initialization_method = 'estimated', initial_level = None, initial_trend = None, initial_seasonal = None, use_boxcox = False, bounds = None, dates = None, freq = None, missing = 'none', smoothing_level = None, smoothing_trend = None, smoothing_seasonal = None, damping_trend = None, ), transforms = (), horizon = 31, )

Модель обучили, теперь можно и предсказания получить. 

Немного поиграемся с переименование колонок, чтобы дальше было удобнее считать метрики

In [238]:
# получаем предсказания 
ts_forecast_df = pipeline.forecast()

# в тестовой выборке целевую переменную переименуем в y_test (это true значения)
forecast_test_df = test_ts_df.to_pandas(flatten=True).rename(columns={"target": "y_test"})

# в датасете с предсказания целевую переменную переименуем в y_pred (это предсказанные значения)
forecast_df = ts_forecast_df.to_pandas(flatten=True).rename(columns={"target": "y_pred"})

# слепим таблицы, чтобы получить общие результаты
hotl_winters_predictions = forecast_test_df.merge(
    forecast_df, 
    on=["timestamp", "segment"], 
    how="outer"
)

# разделяем segment обратно на 3 составляющие 
hotl_winters_predictions["country"] = hotl_winters_predictions["segment"].str.split("_").str[0]
hotl_winters_predictions["store"] = hotl_winters_predictions["segment"].str.split("_").str[1]
hotl_winters_predictions["product"] = hotl_winters_predictions["segment"].str.split("_").str[2]
hotl_winters_predictions = hotl_winters_predictions.drop(columns=["segment"])

# округляем предсказания от модели, т.к. продавать часть пышки - не вариант 
hotl_winters_predictions["y_pred"] = np.ceil(hotl_winters_predictions["y_pred"])#.astype(int)

hotl_winters_predictions.head()

,timestamp,y_test,country_x,product_x,store_x,y_pred,country_y,product_y,store_y,country,store,product
0,2016-12-01,NaN,Canada,Chocolate,Donots,215.0,NaN,NaN,NaN,Canada,Donots,Chocolate
1,2016-12-02,212.0,Canada,Chocolate,Donots,219.0,NaN,NaN,NaN,Canada,Donots,Chocolate
2,2016-12-03,210.0,Canada,Chocolate,Donots,213.0,NaN,NaN,NaN,Canada,Donots,Chocolate
3,2016-12-04,231.0,Canada,Chocolate,Donots,233.0,NaN,NaN,NaN,Canada,Donots,Chocolate
4,2016-12-05,NaN,Canada,Chocolate,Donots,215.0,NaN,NaN,NaN,Canada,Donots,Chocolate


In [239]:
hotl_winters_predictions.shape

(2728, 12)

Отрисуем полученные предсказания в сравнении с истинными значениями на графике. 

In [240]:
@widgets.interact(
    country=widgets.Dropdown(options=hotl_winters_predictions["country"].unique()),
    store=widgets.Dropdown(options=hotl_winters_predictions["store"].unique()),
    products=widgets.Dropdown(options=hotl_winters_predictions["product"].unique()),
    start_date=widgets.DatePicker(value=hotl_winters_predictions["timestamp"].min()),
)
def show_holt_winters_predictions(country: str, store: str, products: str, start_date: date):
    start_date = pd.Timestamp(start_date)
    plot_df = hotl_winters_predictions[
        (hotl_winters_predictions["country"] == country)
        & (hotl_winters_predictions["store"] == store)
        & (hotl_winters_predictions["product"] == products)
        & (hotl_winters_predictions["timestamp"].dt.date >= start_date.date())
    ]

    fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(15, 4))

    axs.plot(plot_df["timestamp"], plot_df["y_pred"], marker="o", label="pred")
    # уберём пропущенные значения, чтобы точки соединились между собой - во имя субъктивного прекрасного 
    plot_df = plot_df.dropna()
    axs.plot(plot_df["timestamp"], plot_df["y_test"], marker="o", label="true")
    axs.set_xlabel("Дата")
    axs.set_ylabel("Кол-во проданных пышек")
    axs.legend()

    plt.show()

interactive(children=(Dropdown(description='country', options=('Canada', 'Finland', 'Italy', 'Kenya', 'Norway'…

Чтобы не потерять результаты работы модели, сохраним предсказания в файл.

In [241]:
pred_dpath = Path("forecast_predictions")
pred_dpath.mkdir(parents=True, exist_ok=True)

pred_fpath = pred_dpath / "holt_winters_predictions.csv"
hotl_winters_predictions.to_csv(pred_fpath)

## Модели регрессоры

Для прогнозирования можно использовать те же модели, что и в "сырой" регрессии. Например, линейную регрессию или LGBM. Их мы и поварим для наших пышек.

Но есть большое жирное **НО**. Здесь мы уже не сможем использовать ETNA, и все признаки нужно будет сгенерировать ручками. Что ж, погнали! 

### Генерация признаков

Признаки нужны будут как для обучающей выборки, так и для тестовой. Поэтому объединим их обратно в единую таблицу, сгенерим признаки, а потом снова разделим по той же дате 1 декабря 2016. 

In [242]:
df = pd.concat((train_df, test_df)).set_index("timestamp")

df.head()

,country,store,product,target,segment
timestamp,,,,,
2010-01-01,Canada,Donots,Chocolate,300.0,Canada_Donots_Chocolate
2010-01-02,Canada,Donots,Chocolate,281.0,Canada_Donots_Chocolate
2010-01-03,Canada,Donots,Chocolate,297.0,Canada_Donots_Chocolate
2010-01-04,Canada,Donots,Chocolate,235.0,Canada_Donots_Chocolate
2010-01-05,Canada,Donots,Chocolate,240.0,Canada_Donots_Chocolate


Нет единого верного способа какие признаки генерить и как, тут вас ограничивает только ваша фантазия и доступные данные. 

Мы создадим три типа признаков: 
- признаки с информацией о днях;
- признаки о праздниках;
- признаки о "лагах".

<details>
    <summary>🤓 Что такое лаги? [Нажми на меня]</summary>

**Лаги** - это какие-то предыдущие значения временного ряда. Например, первый лаг - это вчерашнее значение, а пятый лаг - это значение 5 дней назад и т.д. Такие признаки очень важны для регрессионных моделей, т.к. им нужно получить информацию о прошлом ряда. 

</details>

Генерить признаки (фичи) нужно для каждого ряда в отдельности, т.к. для каждого ряда мы будем учить свою собственную модель. В итоге у нас получится 90 моделей регрессии (на самом деле 88, т.к. 2 пустых ряда мы выкинули). А не одна, как в ETNA. 

Давайте для примера посмотрим генерацию признаков для одного ряда, а потом напишем цикл (пайплайн), который будет проходить по каждому ряду. 

Допустим выберем кофейную пышку в пышечной Pyshka в Сингапуре.

In [243]:
SAMPLE_COUNTRY = "Singapore"
SAMPLE_STORE = "Pyshka"
SAMPLE_PRODUCT = "Coffee"

In [244]:
SAMPLE_COUNTRY = "Singapore"
SAMPLE_STORE = "Pyshka"
SAMPLE_PRODUCT = "Coffee"

# TODO - отфильтруйте только данные для одного ряда

sample_df = train_df[
    (train_df["country"] == SAMPLE_COUNTRY) & 
    (train_df["store"] == SAMPLE_STORE) & 
    (train_df["product"] == SAMPLE_PRODUCT)
]

sample_df.head()

,timestamp,country,store,product,target,segment
212184,2010-01-01,Singapore,Pyshka,Coffee,1045.0,Singapore_Pyshka_Coffee
212185,2010-01-02,Singapore,Pyshka,Coffee,1010.0,Singapore_Pyshka_Coffee
212186,2010-01-03,Singapore,Pyshka,Coffee,1040.0,Singapore_Pyshka_Coffee
212187,2010-01-04,Singapore,Pyshka,Coffee,846.0,Singapore_Pyshka_Coffee
212188,2010-01-05,Singapore,Pyshka,Coffee,770.0,Singapore_Pyshka_Coffee


In [245]:
sample_df.head()

,timestamp,country,store,product,target,segment
212184,2010-01-01,Singapore,Pyshka,Coffee,1045.0,Singapore_Pyshka_Coffee
212185,2010-01-02,Singapore,Pyshka,Coffee,1010.0,Singapore_Pyshka_Coffee
212186,2010-01-03,Singapore,Pyshka,Coffee,1040.0,Singapore_Pyshka_Coffee
212187,2010-01-04,Singapore,Pyshka,Coffee,846.0,Singapore_Pyshka_Coffee
212188,2010-01-05,Singapore,Pyshka,Coffee,770.0,Singapore_Pyshka_Coffee


In [246]:
assert set(sample_df["country"].unique()) == set([SAMPLE_COUNTRY]), "Должен быть только Сингапур в данных"
assert set(sample_df["store"].unique()) == set([SAMPLE_STORE]), "Должна быть только одна пышечная в данных"
assert set(sample_df["product"].unique()) == set([SAMPLE_PRODUCT]), "Должна быть только кофейная пышка в данных"

print("Тесты прошли! Всё хорошо!")

Тесты прошли! Всё хорошо!


Оставим только информацию о дате и целевую колонку. Остальные колонки содержат повторяющиеся значения и модели они ничем не помогут.

In [247]:
sample_df = sample_df[["target"]]
sample_df.head()

,target
212184,1045.0
212185,1010.0
212186,1040.0
212187,846.0
212188,770.0


Начнём с информации о днях. Сюда относится такая информация как день месяца, день недели, четверть года, к которому принадлежит текущий timestamp, порядковый день года, порядковая неделя года и т.д и т.п. Всё, что вы только можете придумать.  

Давайте сделаем отдельную функцию для генерации таких фич, она нам пригодится ещё в пайплайне.

In [ ]:
import pandas as pd
import numpy as np


def create_dtime_features(df: pd.DataFrame) -> pd.DataFrame:
    """创建时间相关特征（依赖DataFrame的datetime索引）"""
    df_new = df.copy()
    
    
    df_new["dayofweek"] = df_new.index.dayofweek.tolist()
    df_new["cos_weekday"] = np.cos(df_new["dayofweek"] / 7 * 2 * np.pi)
    df_new["sin_weekday"] = np.sin(df_new["dayofweek"] / 7 * 2 * np.pi)
    
    
    df_new["is_leap_year"] = df_new.index.is_leap_year.tolist()
    df_new["dayofyear"] = df_new.index.dayofyear.tolist()
    df_new["cos_day"] = np.cos(df_new["dayofyear"] / (365 + df_new["is_leap_year"]) * 2 * np.pi)
    df_new["sin_day"] = np.sin(df_new["dayofyear"] / (365 + df_new["is_leap_year"]) * 2 * np.pi)
    
   
    df_new["quarter"] = df_new.index.quarter.tolist()
    df_new["year"] = df_new.index.year.tolist()
    df_new["month"] = df_new.index.month.tolist()
    df_new["day"] = df_new.index.day.tolist()
    df_new["weekofyear"] = df_new.index.isocalendar().week.tolist()
    
    
    df_new["is_weekend"] = 0
    df_new.loc[df_new["dayofweek"].isin([5, 6]), "is_weekend"] = 1
    
    
    df_new["is_month_start"] = df_new.index.is_month_start.tolist()
    df_new["is_month_end"] = df_new.index.is_month_end.tolist()
    
    return df_new



dates = pd.date_range(start="2025-01-01", periods=5, freq="D")
sample_df = pd.DataFrame({
    "sales": [100, 120, 90, 110, 130]
}, index=dates)



sample_df = create_dtime_features(sample_df)
sample_df.head()

,sales,dayofweek,cos_weekday,sin_weekday,is_leap_year,dayofyear,cos_day,sin_day,quarter,year,month,day,weekofyear,is_weekend,is_month_start,is_month_end
2025-01-01,100,2,-0.222521,0.974928,False,1,0.999852,0.017213,1,2025,1,1,1,0,True,False
2025-01-02,120,3,-0.900969,0.433884,False,2,0.999407,0.034422,1,2025,1,2,1,0,False,False
2025-01-03,90,4,-0.900969,-0.433884,False,3,0.998667,0.051620,1,2025,1,3,1,0,False,False
2025-01-04,110,5,-0.222521,-0.974928,False,4,0.997630,0.068802,1,2025,1,4,1,1,False,False
2025-01-05,130,6,0.623490,-0.781831,False,5,0.996298,0.085965,1,2025,1,5,1,1,False,False


In [249]:
# Вызываем функцию и посмотрим, что у нас стало с табличкой
sample_df = create_dtime_features(sample_df)

sample_df.head()

,sales,dayofweek,cos_weekday,sin_weekday,is_leap_year,dayofyear,cos_day,sin_day,quarter,year,month,day,weekofyear,is_weekend,is_month_start,is_month_end
2025-01-01,100,2,-0.222521,0.974928,False,1,0.999852,0.017213,1,2025,1,1,1,0,True,False
2025-01-02,120,3,-0.900969,0.433884,False,2,0.999407,0.034422,1,2025,1,2,1,0,False,False
2025-01-03,90,4,-0.900969,-0.433884,False,3,0.998667,0.051620,1,2025,1,3,1,0,False,False
2025-01-04,110,5,-0.222521,-0.974928,False,4,0.997630,0.068802,1,2025,1,4,1,1,False,False
2025-01-05,130,6,0.623490,-0.781831,False,5,0.996298,0.085965,1,2025,1,5,1,1,False,False


Пуф! И у нас уже 15 новых признаков (колонок) 😎 Дальше больше! 

Теперь добавим признаки о праздниках. Есть радостная новость, праздники не нужно добавлять руками, за нас это уже сделали другие люди в библиотеке `Prophet`. 

Достаточно указать страну и диапазон лет. И получим исчерпывающий список праздников.

Для удобства модели закодируем бинарным значением каждый праздник.

In [250]:
def get_holiday_df(country_name: str) -> pd.DataFrame:
    holidays_df = make_holidays_df(year_list=range(2010, 2017), country=country_name)

    holidays_df["is_holiday"] = 1
    holidays_df = holidays_df.rename(columns={"ds": "timestamp"})
    holidays_df = holidays_df.set_index("timestamp")

    return holidays_df

In [251]:
sample_holidays_df = get_holiday_df(SAMPLE_COUNTRY)

sample_holidays_df.head()

,holiday,is_holiday
timestamp,,
2016-01-01,New Year's Day,1
2016-02-08,Chinese New Year,1
2016-02-09,Chinese New Year,1
2016-07-06,Eid al-Fitr,1
2016-09-12,Eid al-Adha,1


Чтобы расширить признаки, добавим в таблицу лаги после праздничного дня (1 и 2 дня после).

Для удобства оформим всё это в отдельную функцию.

In [252]:
def create_holiday_features(df: pd.DataFrame, holidays_df: pd.DataFrame) -> pd.DataFrame:
    """Create holiday features."""
    df_new = df.copy()
    df_new = df_new.merge(holidays_df[["is_holiday"]], left_index=True, right_index=True, how="outer")
    df_new["is_holiday"] = df_new["is_holiday"].fillna(0)
    df_new["holiday_lag_1"] = df_new["is_holiday"].shift(1)
    df_new["holiday_lag_2"] = df_new["is_holiday"].shift(2)
    return df_new

In [253]:
sample_df = create_holiday_features(sample_df, sample_holidays_df)

sample_df.head()

,sales,dayofweek,cos_weekday,sin_weekday,is_leap_year,dayofyear,cos_day,sin_day,quarter,year,month,day,weekofyear,is_weekend,is_month_start,is_month_end,is_holiday,holiday_lag_1,holiday_lag_2
2010-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
2010-02-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN
2010-02-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0
2010-02-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0
2010-04-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0


Ещё +3 признака. Итого уже 18 признаков для обучения модели.

Переходим к "лагнутым" признакам. Сгенерим лаги на 7, 10 и 15 дней назад. 

Помимо этого добавим ещё немного статистики по лагам: скользящее среднее значение с окном 14 и 7 дней. 

Опять же какие лаги выбрать, чтобы модели помочь, никто не скажет. Тут нужно экспериментировать и отталкиваться от задачи и данных.

Для удобства обернём всё в функцию.

In [ ]:
def create_lag_features(df: pd.DataFrame, horizon: int) -> pd.DataFrame:
    """Create lag features for time series."""
    df_new = df.copy()
    
    
    if 'target' not in df_new.columns:
       
        possible_targets = ['num_sold', 'sales', 'value']  
        target_col = None
        for col in possible_targets:
            if col in df_new.columns:
                target_col = col
                break
        
        if target_col is None:
            
            numeric_cols = df_new.select_dtypes(include=[np.number]).columns
            if len(numeric_cols) > 0:
                target_col = numeric_cols[0]
            else:
                raise KeyError("找不到目标列，请确保数据包含数值型目标变量")
        
       
        df_new = df_new.rename(columns={target_col: 'target'})
    
    
    for lag in range(1, horizon + 1):
        df_new[f"lag_{lag}"] = df_new["target"].shift(lag)
    
   
    for window in [7, 14, 30]:
        df_new[f"rolling_mean_{window}"] = df_new["target"].shift(1).rolling(window=window).mean()
        df_new[f"rolling_std_{window}"] = df_new["target"].shift(1).rolling(window=window).std()
        df_new[f"rolling_min_{window}"] = df_new["target"].shift(1).rolling(window=window).min()
        df_new[f"rolling_max_{window}"] = df_new["target"].shift(1).rolling(window=window).max()
    
    return df_new

In [255]:
sample_df = create_lag_features(sample_df, horizon=HORIZON)
sample_df.head()

,target,dayofweek,cos_weekday,sin_weekday,is_leap_year,dayofyear,cos_day,sin_day,quarter,year,...,rolling_min_7,rolling_max_7,rolling_mean_14,rolling_std_14,rolling_min_14,rolling_max_14,rolling_mean_30,rolling_std_30,rolling_min_30,rolling_max_30
2010-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-02-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-02-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-02-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-04-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [256]:
sample_df.tail()

,target,dayofweek,cos_weekday,sin_weekday,is_leap_year,dayofyear,cos_day,sin_day,quarter,year,...,rolling_min_7,rolling_max_7,rolling_mean_14,rolling_std_14,rolling_min_14,rolling_max_14,rolling_mean_30,rolling_std_30,rolling_min_30,rolling_max_30
2025-01-01,100.0,2.0,-0.222521,0.974928,False,1.0,0.999852,0.017213,1.0,2025.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-01-02,120.0,3.0,-0.900969,0.433884,False,2.0,0.999407,0.034422,1.0,2025.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-01-03,90.0,4.0,-0.900969,-0.433884,False,3.0,0.998667,0.051620,1.0,2025.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-01-04,110.0,5.0,-0.222521,-0.974928,False,4.0,0.997630,0.068802,1.0,2025.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-01-05,130.0,6.0,0.623490,-0.781831,False,5.0,0.996298,0.085965,1.0,2025.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Важно отметить, что в начале нашей выборки часть лагнутых признаков содержит `NaN` в ячейках. Это случилось потому, что это начало истории и лаги просто неоткуда взять. При обучении эти строки придётся дропнуть.

Но так как это начало обучающей выборки, то не страшно. Тем более уйдёт всего около 30 строчек, что меньше 1% процента наших данных.

In [257]:
sample_df.shape

(96, 62)

Итак ... лёгким движеним руки у нас уже 63 признака. А помните с чего мы начинали? 1 колонка с днём и 1 целевая колонка.  

Можно добавить и больше признаков, но пока остановимся на этом.

Тут как и везде важно **не переборщить**. Когда признаков очень много (больше 100), то моделям уже сложно разделить их в пространстве и эффективность обучения снижается. 

Если бы у нас был всего 1 ряд, мы бы полученную таблицу разделили обратно на две выборки и обучили бы 1 модель. 

Но так как у нас есть ещё 87 рядов, то давайте напишем цикл, в котором для каждого ряда повторим эти шаги, обучим модель и получим предсказания.

In [258]:
# функция для форматирования полученных предсказаний, чтобы формат таблиц был такой же как у ETNA 
def create_preds_df(y_test, y_pred, country, store, product):
    y_test = y_test.rename(columns={"target": "y_test"})

    y_test["y_pred"] = y_pred
    # округляем предсказания, пышки по частям мы не продаём
    y_test["y_pred"] = np.ceil(y_test["y_pred"])

    y_test["country"] = country
    y_test["store"] = store
    y_test["product"] = product
    preds_df = y_test.reset_index()
    preds_df = preds_df[["timestamp", "y_test", "y_pred", "country", "store", "product"]]

    return preds_df

Собираем всё в большую и красивую кучку.

In [259]:
def run_pipeline(model) -> pd.DataFrame:
    # выделяем уникальные страны в данных 
    countries = df["country"].unique()

    # выделяем уникальные пышечные в данных
    stores = df["store"].unique()

    # выделяем уникальные вкусы пышек в данных
    products = df["product"].unique()

    # кол-во комбинаций, чисто для красивых логов 
    iter_n = len(countries) * len(stores) * len(products)

    missing_predictions = []
    predictions = []

    iter_counter = 1

    # итерируемся по всем комбинациям страна - пышечная - вкус 
    for country in countries:
        for store in stores:
            for product in products:
                print(f"{iter_counter}/{iter_n} {country!r} - {store!r} - {product!r} processing ...")

                # выбираем только 1 временной ряд
                sample_df = df[
                    (df["country"] == country) 
                    & (df["store"] == store)
                    & (df["product"] == product)
                ]
                nan_target_index = sample_df[sample_df["target"].isna()].index
                sample_df = sample_df[["target"]].reset_index()

                if sample_df.empty:
                    missing_predictions.append((country, store, product))
                    continue
                
                sample_df = sample_df.resample("1D", on="timestamp").sum()
                sample_df.index = pd.to_datetime(sample_df.index)

                # генерим временные признаки
                sample_df = create_dtime_features(sample_df)

                # генерим праздничные признаки 
                sample_holidays_df = get_holiday_df(country)
                sample_df = create_holiday_features(sample_df, sample_holidays_df)
                
                # генерим лагнутые признаки 
                sample_df = create_lag_features(sample_df, horizon=HORIZON)

                # делим данные обратно на 2 выборки: train и test

                # обучающая выборка
                sample_train_df = sample_df.loc[: SPLIT_DATE - timedelta(days=1)]
                # дропнем строки, где есть пропущенные значения 
                sample_train_df = sample_train_df.dropna()

                # тестовая выборка
                sample_test_df = sample_df.loc[SPLIT_DATE:]

                # разделяем выборки на признаки и целевую переменную
                X_train = sample_train_df.drop(columns=["target"])
                y_train = sample_train_df["target"]

                X_test = sample_test_df.drop(columns=["target"])
                # X_test = X_test.fillna(method="bfill").fillna(method="ffill")

                y_test = sample_test_df[["target"]]
                nan_y_test_index = set(y_test.index) & set(nan_target_index)
                y_test.loc[nan_y_test_index, "target"] = None

                iter_counter += 1

                # на всякий случай проверяем данные на "пустоту"
                if X_train.empty or X_test.empty:
                    missing_predictions.append((country, store, product))
                    continue

                # обучаем модель
                model.fit(X_train, y_train)
                # получаем предсказания
                y_pred = model.predict(X_test)

                # форматируем предсказания в удобный нам формат
                preds_df = create_preds_df(y_test, y_pred, country, store, product)
                predictions.append(preds_df)

    # объединяем предсказания ото всех рядов в единую таблицу
    predictions_df = pd.concat(predictions)
    return predictions_df

Все приготовления сделаны, погнали учить модельки! 

### Линейная регрессия

Первой будет старая добрая модель линейной регрессии.

In [260]:
linreg_model = LinearRegression()
linreg_predictions = run_pipeline(linreg_model)

linreg_predictions.head()

1/90 'Canada' - 'Donots' - 'Chocolate' processing ...
2/90 'Canada' - 'Donots' - 'Coffee' processing ...
3/90 'Canada' - 'Donots' - 'Cream' processing ...
4/90 'Canada' - 'Donots' - 'Strawberry Jam' processing ...
5/90 'Canada' - 'Donots' - 'Sugar Powder' processing ...
6/90 'Canada' - 'Fluffy Bounce' - 'Chocolate' processing ...
6/90 'Canada' - 'Fluffy Bounce' - 'Coffee' processing ...
7/90 'Canada' - 'Fluffy Bounce' - 'Cream' processing ...
8/90 'Canada' - 'Fluffy Bounce' - 'Strawberry Jam' processing ...
9/90 'Canada' - 'Fluffy Bounce' - 'Sugar Powder' processing ...
10/90 'Canada' - 'Pyshka' - 'Chocolate' processing ...
11/90 'Canada' - 'Pyshka' - 'Coffee' processing ...
12/90 'Canada' - 'Pyshka' - 'Cream' processing ...
13/90 'Canada' - 'Pyshka' - 'Strawberry Jam' processing ...
14/90 'Canada' - 'Pyshka' - 'Sugar Powder' processing ...
15/90 'Finland' - 'Donots' - 'Chocolate' processing ...
16/90 'Finland' - 'Donots' - 'Coffee' processing ...
17/90 'Finland' - 'Donots' - 'Cream' p

,timestamp,y_test,y_pred,country,store,product
0,2016-12-01,NaN,220.0,Canada,Donots,Chocolate
1,2016-12-02,212.0,173.0,Canada,Donots,Chocolate
2,2016-12-03,210.0,208.0,Canada,Donots,Chocolate
3,2016-12-04,231.0,245.0,Canada,Donots,Chocolate
4,2016-12-05,NaN,228.0,Canada,Donots,Chocolate


In [261]:
linreg_predictions.shape

(2728, 6)

Также как с моделью ETNA отрисуем на графике сравнение целевых значений и предсказанных моделью.

In [262]:
@widgets.interact(
    country=widgets.Dropdown(options=linreg_predictions["country"].unique()),
    store=widgets.Dropdown(options=linreg_predictions["store"].unique()),
    products=widgets.Dropdown(options=linreg_predictions["product"].unique()),
    start_date=widgets.DatePicker(value=linreg_predictions["timestamp"].min()),
)
def show_linreg_predictions(country: str, store: str, products: str, start_date: date):  # noqa: D103
    start_date = pd.Timestamp(start_date)
    plot_df = linreg_predictions[
        (linreg_predictions["country"] == country)
        & (linreg_predictions["store"] == store)
        & (linreg_predictions["product"] == products)
        & (linreg_predictions["timestamp"].dt.date >= start_date.date())
    ]

    fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(15, 4))

    axs.plot(plot_df["timestamp"], plot_df["y_pred"], marker="o", label="pred")
    # уберём пропущенные значения, чтобы точки соединились между собой - чисто для красоты
    plot_df = plot_df.dropna()
    axs.plot(plot_df["timestamp"], plot_df["y_test"], marker="o", label="true")
    axs.set_xlabel("Дата")
    axs.set_ylabel("Кол-во проданных пышек")
    axs.legend()

    plt.show()

interactive(children=(Dropdown(description='country', options=('Canada', 'Finland', 'Italy', 'Kenya', 'Norway'…

И сохраняем в файл, чтобы не потерять.

In [263]:
pred_dpath = Path("forecast_predictions")
pred_dpath.mkdir(parents=True, exist_ok=True)

pred_fpath = pred_dpath / "linear_regression_predictions.csv"
linreg_predictions.to_csv(pred_fpath)

### LGBM : Бустинг

Для сравнения обучим модель градиентного бустинга.

In [264]:
lgbm_model = LGBMRegressor(random_state=42)
lgbm_predictions = run_pipeline(lgbm_model)

lgbm_predictions.head()

1/90 'Canada' - 'Donots' - 'Chocolate' processing ...
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001280 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 11460
[LightGBM] [Info] Number of data points in the train set: 2495, number of used features: 61
[LightGBM] [Info] Start training from score 223.419258
2/90 'Canada' - 'Donots' - 'Coffee' processing ...
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001066 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 11464
[LightGBM] [Info] Number of data points in the train set: 2495, number of used features: 61
[LightGBM] [Info] Start training from score 655.612425
3/90 'Canada' - 'Donots' - 'Cream' processing ...
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000952 seconds.
You can set `force_col_wise=true` to remove the overh

,timestamp,y_test,y_pred,country,store,product
0,2016-12-01,NaN,215.0,Canada,Donots,Chocolate
1,2016-12-02,212.0,221.0,Canada,Donots,Chocolate
2,2016-12-03,210.0,226.0,Canada,Donots,Chocolate
3,2016-12-04,231.0,247.0,Canada,Donots,Chocolate
4,2016-12-05,NaN,219.0,Canada,Donots,Chocolate


In [265]:
lgbm_predictions.shape

(2728, 6)

Угадайте, что? Отрисуем предсказания. 

In [266]:
@widgets.interact(
    country=widgets.Dropdown(options=lgbm_predictions["country"].unique()),
    store=widgets.Dropdown(options=lgbm_predictions["store"].unique()),
    products=widgets.Dropdown(options=lgbm_predictions["product"].unique()),
    start_date=widgets.DatePicker(value=lgbm_predictions["timestamp"].min()),
)
def show_lgbm_predictions(country: str, store: str, products: str, start_date: date):  # noqa: D103
    start_date = pd.Timestamp(start_date)
    plot_df = lgbm_predictions[
        (lgbm_predictions["country"] == country)
        & (lgbm_predictions["store"] == store)
        & (lgbm_predictions["product"] == products)
        & (lgbm_predictions["timestamp"].dt.date >= start_date.date())
    ]

    fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(15, 4))

    axs.plot(plot_df["timestamp"], plot_df["y_pred"], marker="o", label="pred")
    # уберём пропущенные значения, чтобы точки соединились между собой - чисто для красоты
    plot_df = plot_df.dropna()
    axs.plot(plot_df["timestamp"], plot_df["y_test"], marker="o", label="true")
    axs.set_xlabel("Дата")
    axs.set_ylabel("Кол-во проданных пышек")
    axs.legend()

    plt.show()

interactive(children=(Dropdown(description='country', options=('Canada', 'Finland', 'Italy', 'Kenya', 'Norway'…

Сохраняем результаты в файл.

In [267]:
pred_dpath = Path("forecast_predictions")
pred_dpath.mkdir(parents=True, exist_ok=True)

pred_fpath = pred_dpath / "lgbm_predictions.csv"
lgbm_predictions.to_csv(pred_fpath)

Ура! Модели обучены, теперь осталось самое вкусное - посчитать метрики, сравнить модели и сделать выводы. 

Этим займёмся в следующем ноутбуке. 

## Заключение

В этом ноутбуке вы познакомились со статистической моделью с пафосным названием Хольта-Винтерса. 

Посмотрели как натянуть знакомые модели регрессии на прогнозирование. 

И погрузились в удивительный мир генерации признаков. 

## Вопросы для закрепления

1. В чём разница между трендом и сезонностью? 
2. Может ли быть так, что в ряду есть тренд, но нет сезонности? 
3. Можно ли обучить регрессионную модель без лагнутых признаков? 
4. Как обучить регрессионную модель, если в данных несколько временных рядов? 
5. Что нужно сделать, чтобы обучить модель Хольта-Винтерса, если в данных несколько временных рядов? 